# OBTENCIÓN DE LOS DATOS

Para la obtención de los datos utilizaremos la técnica de webscraping en sitios web especializados de buscadores de viviendas prefabricadas como: 
- [Facilhouse](https://facilhouse.com/)

## Obtención de datos de la página web: [Facilhouse](https://facilhouse.com/)

In [1]:
#importamos las librerías necesarias
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs4
import requests

In [41]:
#Definición de listas globales: 
precios = []
metros_cuadrados = []
baños = []
habitaciones = []
material = []
modelo = []
empresa = []

#Esta función se encarga de leer el archivo .txt que contiene el código HTML de la página de resultados Fotocasa. Utiliza BeautifulSoup (from bs4 import BeautifulSoup) para analizar el HTML y devolver un objeto soup, que permite navegar fácilmente por el árbol de etiquetas HTML.
def obtener_soup(nombre_txt):
    with open(nombre_txt) as fp:
        return bs4(fp)

#Esta función recorre todos los elementos del HTML que contienen el precio de un terreno. Si el texto del precio no es "Consultar", lo agrega a la lista precios

def obtener_precios(soup):
    for precio in soup.select("span.item-price"):
        precios.append(precio.text)

#Esta función extrae los metros cuadrados de la propiedad.
def obtener_metros_cuadrados(soup):
    for metro_cuadrado in soup.select("span.h-area"):
            metros_cuadrados.append(metro_cuadrado.text)

#Esta función extrae el número de baños de la propiedad, observamos que hay valores tipo "Hasta 3" los añadimos poniendo el número correspondiente: 3
def obtener_banios(soup):
    for banios in soup.select("span.h-baths"):
        if banios.text == ": Hasta 3":
            baños.append(3)
        else: 
            baños.append(banios.text)
        
#Esta función extrae el número de habitaciones de la propiedad. Hacemos lo mismo que en baños. 
def obtener_habitaciones(soup):
    for habitacion in soup.select("span.h-beds"):
        if habitacion.text == ": Hasta 5":
            habitaciones.append(5)
        else:
            habitaciones.append(habitacion.text)

#Esta función extrae el tipo de material de cada propiedad.
def obtener_material(soup):
    for materiales in soup.select("div.materiales-flex"):
        material.append(materiales.text)

#Esta función extrae el modelo de cada propiedad.
def obtener_modelo(soup):
    for modelos in soup.select("div.property-title-flex"):
        modelo.append(modelos.text)

#Esta función extrae la empresa que vende cada propiedad.
def obtener_empresa(soup):
    for empresas in soup.select("div.empresa-flex.prop-user-agent"):
        empresa.append(empresas.text)


for i in range(1, 66):
    soup = obtener_soup(f"../html/pagina_{i:02}.txt")
    obtener_precios(soup)
    obtener_metros_cuadrados(soup)
    obtener_banios(soup)
    obtener_habitaciones(soup)
    obtener_material(soup)
    obtener_modelo(soup)
    obtener_empresa(soup)


In [42]:
print (len(precios))
print(len(metros_cuadrados))
print(len(baños))
print(len(habitaciones))
print(len(material))
print(len(modelo))
print(len(empresa))

722
721
721
718
722
722
722


## Rellenar variables restantes con Nans
Podemos observar que los datos no tienen la misma longuitud, por lo que para crear el dataframe, rellenaremos con nans las listas (metros_cuadrados, baños y habitaciones)

In [43]:
#Creamos una función para rellenar las variables faltantes con nans, para así crear un dataframe con las mismas longuitudes

longitud_maxima = len(precios)

#creamos una función para rellenar con Nans

def rellenar_nans (lista, longitud_maxima):
     return lista + [np.nan] * (longitud_maxima - len(lista))
 
#rellenamos las listas

metros_cuadrados_1 = rellenar_nans(metros_cuadrados, longitud_maxima)
habitaciones_1= rellenar_nans(habitaciones, longitud_maxima)
baños_1 = rellenar_nans(baños, longitud_maxima)

#Verificamos la longuitud para poder crear el DF
print(len(metros_cuadrados_1))
print(len(habitaciones_1))
print(len(baños_1))

722
722
722


## Limpieza de listas
Una vez hemos obtenido los datos, pasaremos a limpiarlos para añadirlos al DF

In [44]:
#Quitamos el simbolo de euro y el punto a precios
precios_limpio =[
    int(str(price).replace("€", "").replace(".", ""))
    if price and str(price) != "Consultar" else None
    for price in precios
]

#Quitamos m² a metros cuadrados
metros_cuadrados_10 = [
    int(str(metro).replace("m²", "").replace(".", "")) 
    if metro and str(metro) != 'nan' else None 
    for metro in metros_cuadrados_1
]

#Quitamos (:) a baños
baños_limpio = [
    int(str(bath).replace(":", "").replace(" ", ""))
    if bath and str(bath) != "nan" else None
    for bath in baños_1]

#Hacemos lo mismo para habitaciones 
habitaciones_limpio= [
    int(str(hab).replace(":", "").replace(" ", "")) 
    if hab and str(hab) != "nan" else None
    for hab in habitaciones_1]

#Podemos observar que la lista material tiene caracteres especiales como saltos de linea, espacios en blanco etc que no queremos, los eliminamos para quedarnos sólo con el texto
import re
material_limpio = [re.sub(r'[\n\xa0]+', '', item).strip() for item in material]
material_limpio_1 = [item.strip('|').strip() for item in material_limpio]
material_limpio_2 = [mat.replace("+", ",").replace(" ", "") for mat in material_limpio_1]

#Limpiamos ahora la lista modelo quitándole los saltos de linea
modelo_limpio = [item.strip() for item in modelo]

#Hacemos lo mismo para empresa
empresa_limpio = [item.strip() for item in empresa]

## Creación del DataFrame

Una vez hemos limpiado todas las listas, las pasamos a un DataFrame

In [50]:
df = pd.DataFrame({"metros_cuadrados": metros_cuadrados_10,
                   "baños": baños_limpio,
                   "habitaciones": habitaciones_limpio,
                   "material": material_limpio_2,
                   "modelo": modelo_limpio,
                   "empresa": empresa_limpio,
                   "precio": precios_limpio
    
})

df.head(5)

,metros_cuadrados,baños,habitaciones,material,modelo,empresa,precio
0,73.0,2.0,2.0,"acero,hormigón,madera",Modelo B2H BS,Grupo CUNI,109500.0
1,105.0,1.0,2.0,madera,Casa Prefabricada 105,Norges Hus,NaN
2,185.0,2.0,3.0,"acero,hormigón,madera",Zaero Masía Rustico,ZAEROWORKING SL,297400.0
3,130.0,2.0,3.0,madera,Riekko,Forest House,181300.0
4,161.0,2.0,4.0,madera,Ailo,Forest House,241500.0


In [51]:
df

,metros_cuadrados,baños,habitaciones,material,modelo,empresa,precio
0,73.0,2.0,2.0,"acero,hormigón,madera",Modelo B2H BS,Grupo CUNI,109500.0
1,105.0,1.0,2.0,madera,Casa Prefabricada 105,Norges Hus,NaN
2,185.0,2.0,3.0,"acero,hormigón,madera",Zaero Masía Rustico,ZAEROWORKING SL,297400.0
3,130.0,2.0,3.0,madera,Riekko,Forest House,181300.0
4,161.0,2.0,4.0,madera,Ailo,Forest House,241500.0
...,...,...,...,...,...,...,...
717,298.0,3.0,2.0,madera,Visitante,Casastar,NaN
718,64.0,1.0,NaN,madera,Badajoz 298,Madereco,NaN
719,174.0,2.0,NaN,madera,Casa 64,Madercás,NaN
720,36.0,1.0,NaN,madera,Casa prefabricada 174,Norges Hus,NaN


## Exportamos el DataFrame a un csv

In [52]:
df.to_csv('facil_house.csv', index=False) 